In [1]:
import numpy as np 
import pandas as pd 
import re

In [2]:
import pseudogene_filter_funcstions as func

In [3]:
domain_reference = pd.read_csv('./data/HumanORTMD.txt')

In [4]:
query_file = 'AnaPla_step_4_result_mafftAlignment.fasta'
query = func.read_fasta(f"./data/{query_file}")

In [5]:
# Adds in start and end location of domains for current alignment. 
domain_reference = func.get_domain_info(domain_reference, 
                                        human_seq = query[query['name'] == 'Human_OR2J3'].sequence[0])

In [6]:
ref_seq = query[query['name'].str.contains('Human')].sequence[0]

query['mismatch'] = False
for index, row in query.iterrows():
    for domain in ['TM1', 'TM2', 'TM3', 'TM4', 'TM5', 'TM6', 'TM7']:
        start = int(domain_reference[domain_reference['domain'] == domain].start)
        end = int(domain_reference[domain_reference['domain'] == domain].end)

        # Searching for mismatches between seq 
        mismatch = func.seq_mismatch(ref_seq[start:end],
                                    row.sequence[start:end], 
                                    consecutive_mismatch_threshold = 5)

        if mismatch == True:
            query.loc[query['name'] == row['name'], 'mismatch'] = mismatch
            break
        
        # If there are < 5 aa after the TM7 domain then mismatch also True
        if domain == 'TM7':
            c_term_len = len(row.sequence[end+1:len(row.sequence)].replace('-',''))
            if c_term_len <= 5:
                query.loc[query['name'] == row['name'], 'mismatch'] = True
                

In [7]:
query.to_csv(f"./output/{query_file}", index = 0)

In [8]:
removed_query = func.read_fasta('./data/AnaPla_step_4_result_mafftAlignment_einsi_removed3_realign3.fasta')
removed_query['remove_Niv'] = False

In [9]:
query = pd.merge(query, removed_query[['name', 'remove_Niv']], on=['name'], how = 'outer').replace(np.nan, True)

In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

query[(query.mismatch==True) | (query.remove_Niv==True)]

In [11]:
# Considered non-mismatch but Niv removed 
query[(query.mismatch==True) & (query.remove_Niv==False)]

,name,sequence,mismatch,remove_Niv
22,AnaPla_JACEUM010000035.1_1492505_1493362_-1::J...,----------------------------------------------...,True,False
130,AnaPla_JACEUM010000035.1_2943761_2944702_1::JA...,----------------------------------------------...,True,False
221,AnaPla_JACEUM010000035.1_653574_654479_-1::JAC...,-------------------------M--------------------...,True,False
433,AnaPla_JACEUM010000034.1_1066488_1067330_-1::J...,----------------------------------------------...,True,False
984,AnaPla_JACEUM010000037.1_808288_809142_1::JACE...,-------------------------M--------------------...,True,False
1103,AnaPla_JACEUM010000036.1_2017899_2018705_1::JA...,----------------------------------------------...,True,False
1193,AnaPla_JACEUM010000036.1_206011_206817_-1::JAC...,----------------------------------------------...,True,False


In [12]:
# Considered non-mismatch but Niv removed 
query[(query.mismatch==False) & (query.remove_Niv==True)]

,name,sequence,mismatch,remove_Niv
2,AnaPla_JACEUM010000035.1_1529835_1530629_-1::J...,-------------------------ML-------------------...,False,True
78,AnaPla_JACEUM010000040.1_33698_34492_1::JACEUM...,-------------------------ML-------------------...,False,True
190,AnaPla_JACEUM010000035.1_3013002_3013946_-1::J...,-------------------------MI-------------------...,False,True
616,AnaPla_JACEUM010000037.1_1221643_1222437_-1::J...,-------------------------M--------------------...,False,True
923,AnaPla_JACEUM010000034.1_1378270_1379064_1::JA...,----------------------------------------------...,False,True
1122,AnaPla_JACEUM010000035.1_1066308_1067117_1::JA...,----------------------------------------------...,False,True
1262,AnaPla_JACEUM010000314.1_41195_41992_1::JACEUM...,-------------------------MY-------------------...,False,True


In [ ]:
len(query[(query.mismatch==False) & (query.remove_Niv==True)])